In [1]:
from fundar import json
import pandas as pd
import decimal
import os
from glob import glob

def round_n(x: float, n: int) -> float:
    return float(round(decimal.Decimal(x), n))

def wrangle(data: dict) -> pd.DataFrame:
    observaciones = list(data[0]['observaciones'].keys())
    barrios_count = len(data[0]['gini_indices'])
    gini_indices = [f'gini_{i}' for i in range(barrios_count)]
    satisfechos = [f'satisfechos_{i}' for i in range(barrios_count)]

    columns = [
        'alpha', 
        'rango_p', 
        'pasos', 
        # *observaciones, (es una serie temporal)
        *satisfechos, 
        *gini_indices, 
        'gini_t'
    ]

    records = {c:[] for c in columns}
    for item in data:
        records['alpha'].append(item['alpha'])
        records['rango_p'].append(round_n(100 * item['rango'], 1))
        records['pasos'].append(item['pasos'])

        for i, satisfecho in enumerate(item['satisfechos']):
            records[f'satisfechos_{i}'].append(satisfecho)

        for i, gini in enumerate(item['gini_indices']):
            records[f'gini_{i}'].append(gini)

        records['gini_t'].append(item['gini_t'])

    df = pd.DataFrame(records, columns=columns)
    df = df.melt(id_vars=['alpha', 'rango_p'], var_name='variable', value_name='valor')

    return df


Data schema:
```python
[
  {
  'observaciones': {
    'utilidad_media': [float], # Tantos como 'pasos'
    'capital_medio': [float]   # Tantos como 'pasos'
    },
   'alpha': float,
   'rango': float,
   'pasos': int,
   'satisfechos': [int],     # tantos como barrios
   'gini_indices': [float],  # tantos como barrios
   'gini_t': float
  } 
]
```

In [ ]:
for file in glob('../results/*.json'):
    #file = '../results/cuadrantes_1.json'
    data = json.load(file)
    df = wrangle(data)
    df.to_csv(file.replace('.json', '.csv'), 
            index=False, 
            sep=',', 
            decimal='.', 
            float_format='%.2f', 
            encoding='utf-8')